# Real Agent Integration with Agentune Simulate

This notebook demonstrates how to integrate a real agent system with the Agentune Simulate library. You'll learn to:

- Implement a custom agent by extending the `Agent` interface
- Integrate your custom agent with the simulation framework
- Run simulations with real agents and simulated customers

## Use Case: Real Agent + Simulated Customer

This pattern is useful for:
- Testing your actual agent system against various customer scenarios
- Evaluating agent performance without human customers
- A/B testing between different agent implementations
- Stress testing your agent with diverse conversation patterns

## Setup and Imports

In [1]:
import os
import getpass
from datetime import datetime

from langchain_core.vectorstores import InMemoryVectorStore
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

from agentune.simulate.models import Conversation, Message, Outcomes
from agentune.simulate.participants.agent.real import RealAgent, RealAgentFactory
from agentune.simulate.rag import conversations_to_langchain_documents
from agentune.simulate.simulation.session_builder import SimulationSessionBuilder
from utils import setup_logging_and_asyncio, load_data_with_outcomes

## API Key Configuration

In [2]:
# Set up OpenAI API key
if not os.getenv("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

print("✓ API key configured")

✓ API key configured


## Environment Setup

In [3]:
# Configure logging and asyncio for Jupyter
setup_logging_and_asyncio()

✓ Logging configured
✓ Asyncio event loop configured for Jupyter


## Load Data and Extract Outcomes

In [4]:
# Load conversations and extract outcomes in one step
conversations, outcomes_tuple = load_data_with_outcomes("data/sample_conversations.csv")
outcomes = Outcomes(outcomes=outcomes_tuple)

Loading conversations from data/sample_conversations.csv...
✓ Loaded 100 conversations
✓ Sample conversation has 6 messages
✓ Extracted 2 unique outcomes
  - resolved: Issue was successfully resolved
  - unresolved: Issue was not resolved


## Create Models and Vector Store

In [5]:
# Setup models and vector store for the simulated customer
chat_model = ChatOpenAI(model="gpt-4o", temperature=0.7)
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
documents = conversations_to_langchain_documents(conversations)
vector_store = InMemoryVectorStore.from_documents(documents, embeddings)

print(f"✓ Loaded {len(conversations)} conversations with vector store")

✓ Loaded 100 conversations with vector store


## Custom Real Agent Implementation

Here we'll create a `MockRealAgent` that simulates calling an external agent system. In practice, this would make HTTP requests, call APIs, or interface with your actual agent platform.

**Important:** Real agents inherit from `RealAgent` instead of `Agent` directly. This provides a sensible default implementation of the `with_intent` method that real agents don't need, creating a cleaner interface for external agent integrations.

In [6]:
class MockRealAgent(RealAgent):
    """A minimal real agent that simulates external agent system integration."""

    def __init__(self):
        self._messages = [
            "Hello! How can I help you today?",
            "I understand your concern. Let me help you with that issue.",
            "Thank you for contacting us. Have a great day!"
        ]
        self._count = 0

    async def get_next_message(self, conversation: Conversation) -> Message | None:
        """Generate agent response - replace this with your actual agent API call."""

        if self._count >= len(self._messages):
            return None  # No more responses. None indicates the agent is not responding.

        agent_response = self._messages[self._count]
        self._count += 1

        return Message(
            sender=self.role,
            content=agent_response,
            timestamp=datetime.now()
        )

print("✓ MockRealAgent class implemented")

✓ MockRealAgent class implemented


In [7]:
# Create a factory for our real agent
class MockRealAgentFactory(RealAgentFactory):
    """Factory for creating MockRealAgent instances."""

    def create_participant(self) -> RealAgent:
        return MockRealAgent()

## Simulation Session with Real Agent

In [8]:
# Create simulation session with real agent + simulated customer
real_agent_factory = MockRealAgentFactory()

session = SimulationSessionBuilder(
    default_chat_model=chat_model,
    outcomes=outcomes,
    vector_store=vector_store,
    max_messages=10
).with_agent_factory(real_agent_factory).build()

print("✓ Simulation session created with real agent")

✓ Simulation session created with real agent


In [9]:
# Run simulation with real agent
base_conversations = conversations[:3]  # Use first 3 conversations as scenarios
result = await session.run_simulation(real_conversations=base_conversations)

print("✓ Real agent simulation completed!")
print(f"Tested on {len(result.simulated_conversations)} conversations")

2025-07-23 10:19:21,539 - Starting intent extraction on 3 conversations
2025-07-23 10:19:26,448 - Finished extracting original intents; generated 3 scenarios
2025-07-23 10:19:26,449 - Starting conversation simulations (self.max_concurrent_conversations=20)
2025-07-23 10:19:31,448 - Progress: 0/3 scenarios completed
2025-07-23 10:19:46,449 - Progress: 3/3 scenarios completed
2025-07-23 10:19:46,449 - Finished simulating conversations; simulated 3 conversations, with 0 failures
2025-07-23 10:19:46,450 - Starting analysis of simulation results
2025-07-23 10:19:53,449 - Finished analyzing results


✓ Real agent simulation completed!
Tested on 3 conversations


## Analyze Results

In [10]:
print(result.generate_summary())

SIMULATION RESULTS
Session name: Simulation Session
Original conversations: 3
Simulated conversations: 3

Average messages per conversation:
  Original: 5.7
  Simulated: 5.7

Outcome distribution comparison:
Outcome              Original        Simulated      
--------------------------------------------------
resolved               3 (100.0%)     0 ( 0.0%)
unresolved             0 ( 0.0%)     3 (100.0%)

Sample conversation (5 messages):
  1. customer: Last night, I waited in line for 2 hours in the business office, but because I only had a copy of my...
  2. agent: Hello! How can I help you today?
  3. customer: I've already explained the issue. I want to cancel my broadband service without unnecessary charges ...
  4. agent: I understand your concern. Let me help you with that issue.
  ... and 1 more messages


## Next Steps

You've successfully integrated a real agent with the Agentune Simulate framework! Here's how to extend this for your use case:

### 1. **Implement Your Real Agent**
Replace the `MockRealAgent` with your actual agent implementation:

```python
class YourRealAgent(RealAgent):
    async def get_next_message(self, conversation: Conversation) -> Optional[Message]:
        # Make API call to your agent system
        response = await your_agent_api.get_response(conversation)
        return Message(sender="agent", content=response, timestamp=datetime.now())
```

### 2. **Use Your Own Data**
Load your own conversations as a list of `Conversation` objects and use them to set up the simulation. Convert your data format to the required structure using utility functions similar to those in `utils.py`.

### 3. **Testing and Validation**
- Use this framework to test your agent against diverse scenarios
- Compare performance metrics between different agent versions

### 4. **Production Deployment**
- Scale simulations with concurrent conversation limits
- Implement caching to lower the number of requests to LLM (simulated customer)
- Monitor agent response patterns and quality metrics
- Set up automated testing pipelines for continuous validation

### 6. **Explore Advanced Features**
Check out the full documentation for more options including:
- Caching of LLM responses for cost efficiency
- LLM failure handling and fallback mechanisms


### Resources:
- [Full Documentation](https://github.com/SparkBeyond/agentune/blob/main/agentune_simulate/README.md)
- [Complete Examples](https://github.com/SparkBeyond/agentune/tree/main/agentune_simulate/examples)
- [Persistent Storage Example](./persistent_storage_example.ipynb) - Full simulation flow, recommended to run before the real agent flow, to benchmark the simulation quality
- [Advanced caching and error handling](https://github.com/SparkBeyond/agentune/tree/main/agentune_simulate/docs/langchain.md)
- [Streamlit Web Interface](https://github.com/SparkBeyond/agentune/blob/main/agentune_simulate/streamlit/README.md) - Visual simulation runner and analyzer